Some more practice with SQLite with `chinook.db`, a database from the Chinook record store, souced from here _(\*link required\*)_

Database schema:

![Chinook database schema image](./chinook-schema.svg)

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT name,
       type
  FROM sqlite_master
 WHERE type IN ("table","view");

* sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Scenario: a deal has been signed with a record label, and 3 new albums out of a potential list of 4 are to be added to Chinook's store. The label specialises in artists from the USA. To determine which ones to pick, an understanding of which genres sell best in the USA should be developed. More precisely, how many tracks have been sold in the USA, per genre, in absolute and percentage numbers?  

The available artists are:

| Artist Name           | Genre     |
| --------------------- | --------- |
| Regal                 | Hip-Hop   |
| Red Tone              | Punk      |
| Meteor and the Girls  | Pop       |
| Slim Jim Bites        | Blues     |

To build up a query to answer this question, the following steps will be performed:

1. Create a temporary view of `customer` querying rows for customers based in the USA.
2. Build on this view by joining `invoice`, `invoice_line`, `track`, and `genre` with LEFT JOINs respectively.
3. Query the above temporary view, aggregating on genre.

In [40]:
%%sql
  WITH usa_customers AS (
       SELECT *
         FROM customer
        WHERE country = 'USA'
       ),
       usa_tracks AS (
       SELECT (uc.first_name || " " || uc.last_name) AS customer_name,
              uc.country,
              i.invoice_id,
              il.invoice_line_id,
              t.track_id,
              t.name AS track_name,
              g.name AS genre
         FROM usa_customers AS uc
              LEFT JOIN invoice AS i
              ON uc.customer_id = i.customer_id
              LEFT JOIN invoice_line AS il
              ON i.invoice_id = il.invoice_id
              LEFT JOIN track AS t
              ON il.track_id = t.track_id
              LEFT JOIN genre AS g
              ON t.genre_id = g.genre_id
       )

SELECT genre,
       COUNT(*) AS tracks_sold_absolute,
       ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM usa_tracks) * 100, 1) AS tracks_sold_perc
  FROM usa_tracks
 GROUP BY genre
 ORDER BY tracks_sold_absolute DESC
 LIMIT 10;

* sqlite:///chinook.db
Done.


genre,tracks_sold_absolute,tracks_sold_perc
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


The above query provides the top 10 genres of tracks sold in the USA. Of the four artists available, the above query suggests the following preference for expected sales performance:

1. Red Tone (Punk)
2. Meteor and the Girls (Pop)
3. Slim Jim Bites (Blues)
4. Regal (Hip Hop)

However, there is only 0.2% difference in track sales between Pop and Hip Hop, so either Slim Jim Bites or Regal could be selected as a 3rd preference.